In [19]:
import dash
from dash import dcc
from dash import html
import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff
import pandas as pd
import numpy as np

# Load the data
file_path = '/Users/aryan.vadera/Desktop/CapstoneProjectA/HealthData.csv'
data = pd.read_csv(file_path)

# Preprocess the data
data['datetime'] = pd.to_datetime(data['date'] + ' ' + data['timestamp'])
data_cleaned = data.copy()
for col in ['heartrate', 'blood_oxygen_level', 'body_temperature', 'air_quality', 'room_temperature', 'hydration_levels']:
    data_cleaned[col] = pd.to_numeric(data_cleaned[col], errors='coerce')
data_cleaned = data_cleaned.dropna()

# Calculate Heart Rate Variability (HRV)
data['hrv'] = data['heartrate'].diff()

# Calculate overall health score (simple average for example)
data['health_score'] = (data['heartrate'] + data['blood_oxygen_level'] + data['body_temperature']) / 3

# Correlation matrix
correlation_matrix = data_cleaned[['heartrate', 'blood_oxygen_level', 'body_temperature', 'air_quality', 'room_temperature', 'hydration_levels']].corr()

# Healthy ranges for elderly people
healthy_ranges = {
    'Heart Rate (BPM)': '60-100',
    'Blood Oxygen Level (%)': '95-100',
    'Body Temperature (°C)': '36.1-37.2',
    'Room Temperature (°C)': '18-21',
    'Hydration Levels (%)': '50-60'
}

# Summary Statistics
summary_stats = {
    'Average Heart Rate (BPM)': round(data['heartrate'].mean(), 2),
    'Average Blood Oxygen Level (%)': round(data['blood_oxygen_level'].mean(), 2),
    'Average Body Temperature (°C)': round(data['body_temperature'].mean(), 2),
    'Average Room Temperature (°C)': round(data['room_temperature'].mean(), 2),
    'Average Hydration Levels (%)': round(data['hydration_levels'].mean(), 2),
    'Total Falls Detected': int(data['fall_detection'].sum()),
    'Health Score': round(data['health_score'].mean(), 2)
}

# Gauge Charts
gauge_fig_heart = go.Figure(go.Indicator(
    mode="gauge+number",
    value=summary_stats['Average Heart Rate (BPM)'],
    title={'text': "Heart Rate (BPM)"},
    gauge={'axis': {'range': [0, 150]},
           'steps': [
               {'range': [60, 100], 'color': "lightgreen"},
               {'range': [100, 120], 'color': "yellow"},
               {'range': [120, 150], 'color': "red"}],
           'bar': {'color': "green"}}
))

gauge_fig_oxygen = go.Figure(go.Indicator(
    mode="gauge+number",
    value=summary_stats['Average Blood Oxygen Level (%)'],
    title={'text': "Blood Oxygen Level (%)"},
    gauge={'axis': {'range': [0, 100]},
           'steps': [
               {'range': [95, 100], 'color': "lightgreen"},
               {'range': [90, 95], 'color': "yellow"},
               {'range': [0, 90], 'color': "red"}],
           'bar': {'color': "green"}}
))

gauge_fig_hydration = go.Figure(go.Indicator(
    mode="gauge+number",
    value=summary_stats['Average Hydration Levels (%)'],
    title={'text': "Hydration Levels (%)"},
    gauge={'axis': {'range': [0, 100]},
           'steps': [
               {'range': [50, 60], 'color': "lightgreen"},
               {'range': [40, 50], 'color': "yellow"},
               {'range': [0, 40], 'color': "red"}],
           'bar': {'color': "green"}}
))

# Creating a list of HTML elements to display the summary with bullet points and improved styling
summary_layout = html.Div([
    html.H3("Summary Statistics", style={
        'text-align': 'center',
        'margin-bottom': '20px',
        'font-family': 'Georgia, serif',
        'font-size': '28px',
        'color': '#2C3E50',
        'text-transform': 'uppercase',
        'letter-spacing': '2px',
        'border-bottom': '2px solid #CCC',
        'padding-bottom': '10px'
    }),
    html.Ul([
        html.Li([html.Span("• ", style={'font-size': '24px', 'color': '#2C3E50'}), f"Average Heart Rate (BPM): {summary_stats['Average Heart Rate (BPM)']}"], style={'margin-bottom': '10px'}),
        html.Li([html.Span("• ", style={'font-size': '24px', 'color': '#2C3E50'}), f"Average Blood Oxygen Level (%): {summary_stats['Average Blood Oxygen Level (%)']}"], style={'margin-bottom': '10px'}),
        html.Li([html.Span("• ", style={'font-size': '24px', 'color': '#2C3E50'}), f"Average Body Temperature (°C): {summary_stats['Average Body Temperature (°C)']}"], style={'margin-bottom': '10px'}),
        html.Li([html.Span("• ", style={'font-size': '24px', 'color': '#2C3E50'}), f"Average Room Temperature (°C): {summary_stats['Average Room Temperature (°C)']}"], style={'margin-bottom': '10px'}),
        html.Li([html.Span("• ", style={'font-size': '24px', 'color': '#2C3E50'}), f"Average Hydration Levels (%): {summary_stats['Average Hydration Levels (%)']}"], style={'margin-bottom': '10px'}),
        html.Li([html.Span("• ", style={'font-size': '24px', 'color': '#2C3E50'}), f"Total Falls Detected: {summary_stats['Total Falls Detected']}"], style={'margin-bottom': '10px'}),
        html.Li([html.Span("• ", style={'font-size': '24px', 'color': '#2C3E50'}), f"Overall Health Score: {summary_stats['Health Score']}"], style={'margin-bottom': '10px'})
    ], style={
        'list-style-type': 'none',
        'font-size': '18px',
        'line-height': '1.6',
        'font-family': 'Arial, sans-serif',
        'color': '#34495E',
        'padding-left': '0'
    })
], style={
    'border': '1px solid #e7e7e7',
    'border-radius': '10px',
    'padding': '30px',
    'box-shadow': '2px 2px 10px rgba(0,0,0,0.1)',
    'background': 'linear-gradient(135deg, #f7f9fc, #e3e7ed)',
    'width': '80%',
    'margin': '0 auto',
    'text-align': 'left'
})

# Healthy range comparison page
healthy_comparison_layout = html.Div([
    html.H3("Comparison with Healthy Standard Ranges (Elderly)", style={
        'text-align': 'center',
        'margin-bottom': '20px',
        'font-family': 'Georgia, serif',
        'font-size': '28px',
        'color': '#2C3E50',
        'text-transform': 'uppercase',
        'letter-spacing': '2px',
        'border-bottom': '2px solid #CCC',
        'padding-bottom': '10px'
    }),
    html.Table([
        html.Tr([html.Th("Metric"), html.Th("Your Average"), html.Th("Healthy Range")]),
        html.Tr([html.Td("Heart Rate (BPM)"), html.Td(f"{summary_stats['Average Heart Rate (BPM)']}"), html.Td(healthy_ranges['Heart Rate (BPM)'])]),
        html.Tr([html.Td("Blood Oxygen Level (%)"), html.Td(f"{summary_stats['Average Blood Oxygen Level (%)']}%"), html.Td(healthy_ranges['Blood Oxygen Level (%)'])]),
        html.Tr([html.Td("Body Temperature (°C)"), html.Td(f"{summary_stats['Average Body Temperature (°C)']}°C"), html.Td(healthy_ranges['Body Temperature (°C)'])]),
        html.Tr([html.Td("Room Temperature (°C)"), html.Td(f"{summary_stats['Average Room Temperature (°C)']}°C"), html.Td(healthy_ranges['Room Temperature (°C)'])]),
        html.Tr([html.Td("Hydration Levels (%)"), html.Td(f"{summary_stats['Average Hydration Levels (%)']}%"), html.Td(healthy_ranges['Hydration Levels (%)'])]),
    ], style={
        'width': '80%',
        'margin': '0 auto',
        'font-size': '18px',
        'border': '1px solid #',
        'border': '1px solid #e7e7e7',
        'border-collapse': 'collapse'
    })
], style={
    'width': '80%',
    'margin': '0 auto',
    'padding': '30px',
    'background': 'linear-gradient(135deg, #f7f9fc, #e3e7ed)',
    'box-shadow': '2px 2px 10px rgba(0,0,0,0.1)',
    'border-radius': '10px'
})

# Gauge charts page
gauge_charts_layout = html.Div([
    html.H3("Gauge Charts for Key Health Metrics", style={
        'text-align': 'center',
        'margin-bottom': '20px',
        'font-family': 'Georgia, serif',
        'font-size': '28px',
        'color': '#2C3E50',
        'text-transform': 'uppercase',
        'letter-spacing': '2px',
        'border-bottom': '2px solid #CCC',
        'padding-bottom': '10px'
    }),
    html.Div([
        dcc.Graph(figure=gauge_fig_heart),
        dcc.Graph(figure=gauge_fig_oxygen),
        dcc.Graph(figure=gauge_fig_hydration)
    ], style={'display': 'flex', 'flex-direction': 'column', 'align-items': 'center'})
], style={
    'width': '80%',
    'margin': '0 auto',
    'padding': '30px',
    'background': 'linear-gradient(135deg, #f7f9fc, #e3e7ed)',
    'box-shadow': '2px 2px 10px rgba(0,0,0,0.1)',
    'border-radius': '10px'
})

# Initialize the Dash app
app = dash.Dash(__name__)

# Defining the layout for the dashboard with improved styles and the new healthy range page
app.layout = html.Div([
    html.H1("Health Monitoring Dashboard", style={
        'text-align': 'center', 
        'font-family': 'Georgia, serif',
        'font-size': '36px', 
        'color': '#34495E',
        'padding-top': '20px',
        'padding-bottom': '20px',
        'letter-spacing': '1.5px',
        'border-bottom': '2px solid #CCC'
    }),
    dcc.Tabs([
        dcc.Tab(label='Summary', children=[summary_layout], style={'padding': '20px'}),
        dcc.Tab(label='Comparison with Healthy Ranges', children=[healthy_comparison_layout], style={'padding': '20px'}),
        dcc.Tab(label='Gauge Charts', children=[gauge_charts_layout], style={'padding': '20px'}),
        dcc.Tab(label='Time-Series Analysis', children=[time_series_summary], style={'padding': '20px'}),
        dcc.Tab(label='Box Plot: Heart Rate by Fall Detection', children=[box_plot_summary], style={'padding': '20px'}),
        dcc.Tab(label='HRV Distribution', children=[hrv_summary], style={'padding': '20px'}),
        dcc.Tab(label='Room vs Body Temperature', children=[temperature_summary], style={'padding': '20px'}),
        dcc.Tab(label='Correlation Heatmap', children=[heatmap_summary], style={'padding': '20px'}),
    ], style={'background-color': '#EAEAEA', 'font-family': 'Arial, sans-serif', 'font-size': '16px', 'border': '1px solid #CCC', 'border-radius': '10px'})
], style={'max-width': '1200px', 'margin': '0 auto', 'padding': '20px', 'background-color': '#FFFFFF', 'box-shadow': '0px 4px 10px rgba(0,0,0,0.1)'})

# Run the app
if __name__ == '__main__':
    app.run(debug=True, port=8052)



## Updated Overview of the Code:

1. **Importing Libraries:**
   * **dash, dash_core_components (dcc), dash_html_components (html)**: Core components of the Dash framework used to build the web app.
   * **plotly.express (px) and plotly.graph_objs (go)**: Libraries for creating interactive visualizations, including high-level plotting (express) and more detailed control (graph_objs).
   * **pandas**: A library for data manipulation and analysis, used for loading and cleaning health data.
   * **numpy**: Used for numerical operations such as computing the difference between heart rate values for HRV.

2. **Loading and Preprocessing the Data:**
   * The code loads a CSV file named `HealthData.csv`.
   * The date and timestamp columns are combined into a single `datetime` column for time-series analysis.
   * Data for key health metrics (e.g., heart rate, blood oxygen level, hydration levels) is converted to numeric format and cleaned by removing missing values.
   * Heart Rate Variability (HRV) is calculated by taking the difference between consecutive heart rate values.

3. **Calculating Key Metrics:**
   * **Overall Health Score**: A simple average is computed from key metrics like heart rate, blood oxygen level, and body temperature, displayed on the summary page.
   * **Healthy Standard Ranges**: Predefined healthy ranges for elderly individuals are included for comparison with actual user data.

4. **Initializing the Dash App:**
   * `app = dash.Dash(__name__)`: Initializes the Dash app.

5. **Creating Interactive Visualizations:**
   * **Summary Page**: Displays a summary of key metrics (heart rate, blood oxygen, body temperature, etc.) with styled bullet points.
   * **Time-Series Analysis**: Shows time-series data for heart rate, blood oxygen, and body temperature, allowing users to see trends over time.
   * **Box Plot**: Visualizes heart rate by fall detection status to compare the heart rate during falls vs. non-falls.
   * **HRV Distribution**: Displays a histogram of heart rate variability, indicating fluctuations in heart rate.
   * **Room vs. Body Temperature**: Compares room temperature and body temperature over time.
   * **Correlation Heatmap**: Shows the relationships between different health metrics using a heatmap.

6. **New Features and Visualizations:**
   * **Gauge Charts for Key Metrics**: A new page/tab was added with gauge charts to visualize heart rate, blood oxygen levels, and hydration levels against healthy ranges. The gauges are oriented vertically to improve readability for elderly users.
   * **Comparison with Healthy Ranges**: A separate page provides a side-by-side comparison between the user's average metrics and healthy standard ranges for elderly people. This helps users understand how their health metrics stack up against optimal values.
   
7. **Creating the Layout:**
   * The `app.layout` defines the overall structure of the dashboard, using `html.Div` to organize the layout and `dcc.Tabs` to switch between different pages (e.g., summary, time-series analysis, gauge charts).
   * Each tab contains visualizations (e.g., graphs, tables) created using Dash components such as `dcc.Graph` and `html.Table`.

8. **Running the App:**
   * The `if __name__ == '__main__':` block ensures that the Dash app runs only when the script is executed directly.
   * `app.run_server(debug=True)` starts the web server, allowing the user to view the dashboard in their web browser.

### Summary of Updates:
- Added **Gauge Charts** for heart rate, blood oxygen level, and hydration, displayed vertically for easier readability.
- Included a new **Comparison Page** to compare user data with healthy ranges for elderly individuals.
- Maintained the flexible layout with multiple tabs for easy navigation between visualizations.
